In [21]:
import cx_Oracle
import requests as rq
from bs4 import BeautifulSoup
import ujson
from tqdm.auto import tqdm
import random
from time import sleep
from os import environ
import traceback

In [ ]:
# Initialize database parameters and connection
environ['TNS_ADMIN'] = r'C:\oracle\instantclient_19_8\adw2_wallet'
lib_dir = r'C:\oracle\instantclient_19_8'

try:
    cx_Oracle.init_oracle_client(lib_dir=lib_dir)
except cx_Oracle.ProgrammingError:
    print(f'Initialization of Oracle Client failed: \n {traceback.format_exc()}')

with open('db_credentials.txt', 'r') as f:
    file_content = f.readline()
    schema = file_content.split(',')[0]
    password = file_content.split(',')[1]
    tns_name = file_content.split(',')[2]

In [2]:
# Get style list
with open('saatchi style list.html') as f:
    html = f.read()

soup = BeautifulSoup(html)
styles_html = soup.findAll('a')
styles = [(x.get('href')).replace('/paintings/', '').replace('?hitsPerPage=100', '') for x in styles_html if x.get('href')]
styles

['abstract',
 'fine-art',
 'modern',
 'abstract-expressionism',
 'expressionism',
 'figurative',
 'impressionism',
 'realism',
 'conceptual',
 'portraiture',
 'surrealism',
 'pop-art',
 'minimalism',
 'illustration',
 'art-deco',
 'street-art',
 'photorealism',
 'folk',
 'cubism',
 'documentary',
 'dada']

In [19]:
def get_random_waittime():
    waittime = random.weibullvariate(1, 5)*2
    return waittime

In [16]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'Accept-Language': 'en,en-US;q=0,5',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8'
}
base_url = 'https://www.saatchiart.com/paintings/'
json_content_list = []
loop_counter = 0

with cx_Oracle.connect(schema, password, tns_name, encoding="UTF-8") as connection:
    cur = connection.cursor()
    sql_stmt = 'insert into saatchi_scrapings_level1 (json_content) ' \
                       'values (:json_content)'

    for style in tqdm(styles[:4]):
        params = {'hitsPerPage': '100',
                  'page': 1,
                  'orientation': 'landscape'}
        url = base_url + style
        print(url)
        page_html = rq.get(url, params=params, headers=headers)
        soup = BeautifulSoup(page_html.content)
        last_page_number = int(soup.find('a', {'data-type': 'last-page'}).get('data-value'))

        with tqdm(total=last_page_number) as pbar:
            while params['page'] <= last_page_number:
                loop_counter = loop_counter + 1
                params.update({'page': params['page'] + 1})
                page_html = rq.get(url, params=params, headers=headers)
                soup = BeautifulSoup(page_html.content)
                page_json_dict = ujson.loads(soup.find('script', {'id': '__NEXT_DATA__'}).contents[0])
                # json_content_list.append(page_json_dict['query']['page']['data']['algoliaResults']['hits']['results'])
                json_blob = str.encode(ujson.dumps(page_json_dict['query']['page']['data']['algoliaResults']['hits']['results']))
                cur.execute(sql_stmt, [json_blob])
                sleep(get_random_waittime())
                pbar.update(1)

                if loop_counter % 10 == 0:
                    connection.commit()

  0%|          | 0/4 [00:00<?, ?it/s]

https://www.saatchiart.com/paintings/abstract


  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
l = []


for i in json_content_list:
    l = l + i
len(l)

'https://www.saatchiart.com/paintings/abstract-expressionism?hitsPerPage=100&page=3'

In [18]:
l

[{'artId': 2638433,
  'artistArtworksUrl': 'https://www.saatchiart.com/account/artworks/420935',
  'artistRecognition': [],
  'artworkId': 2638433,
  'artworkRank': 322,
  'artworkTitle': 'Optical de la serie TRAMAS OPTiCAS',
  'artworkUrl': 'https://www.saatchiart.com/art/Painting-Optical-de-la-serie-TRAMAS-OPTiCAS/420935/2448313/view',
  'aspectRatio': 0.98302263648469,
  'category': 'Paintings',
  'city': 'BUENOS AIRES',
  'color': '#2C1378',
  'colorProximity': 39,
  'country': 'Argentina',
  'curatedStatus': 1,
  'depth': 2,
  'firstName': 'Adriana',
  'hasLimitedEditions': False,
  'hasOpenEditions': True,
  'height': 40,
  'idUser': 420935,
  'imageUrl': 'https://images.saatchiart.com/saatchi/420935/art/2638433/1708326-YEGGUYCZ-6.jpg',
  'isSafe': True,
  'isSeoValid': True,
  'keywords': ['pink',
   'repetition',
   'black',
   'shape',
   'op art',
   'OPTICO ',
   'abstract',
   'geometric',
   'minimal',
   'modern',
   'optical'],
  'lastName': 'Ablin Abstract Painter',
  '